Nielsen data
===

In [1]:
import pandas as pd
import numpy as np

## Loading the data

In [2]:
panelist = pd.read_table("../../Nielsen/panelists_2016.tsv").set_index('Household_Cd')[['Fips_State_Cd', 'Fips_State_Desc', 'Fips_County_Cd', 'Fips_County_Desc', 'Panelist_ZipCd']]
panelist['household_county_fips'] = np.vectorize(int)(panelist.Fips_State_Cd * 1e3 + panelist.Fips_County_Cd)
panelist['household_zip3'] = panelist.Panelist_ZipCd // 100
panelist = panelist[['Fips_State_Desc', 'Fips_County_Desc', 'household_county_fips', 'household_zip3']].rename(columns={'Fips_State_Desc': 'household_state', 'Fips_County_Desc': 'household_county'})

In [3]:
panelist.head()

,household_state,household_county,household_county_fips,household_zip3
Household_Cd,,,,
2058978,CA,ALAMEDA,6001,945
2059041,CA,SONOMA,6097,954
2053746,PA,LANCASTER,42071,175
2053784,PA,LANCASTER,42071,176
2055288,CA,ALAMEDA,6001,945


In [4]:
purchases = pd.read_csv("../../Nielsen/purchases_subset_2016.csv")
purchases['upc_price'] = purchases.total_price_paid / purchases.quantity
purchases = purchases[['trip_code_uc', 'upc', 'upc_ver_uc', 'upc_price']]


In [5]:
purchases.head()

,trip_code_uc,upc,upc_ver_uc,upc_price
0,1074839376,1111040601,1,1.49
1,1067492860,1111040601,1,1.13
2,1073807635,1111040601,1,1.21
3,1066280722,1111040601,1,1.82
4,1074711109,1111040601,1,1.48


In [6]:
trips = pd.read_table("../../Nielsen/trips_2016.tsv", parse_dates=['purchase_date']).set_index('trip_code_uc')[['purchase_date', 'retailer_code', 'store_code_uc', 'store_zip3', 'household_code']]
# colonnes à enlever : panel_year, total_spent, method_of_payment_cd

In [7]:
trips.head()

,purchase_date,retailer_code,store_code_uc,store_zip3,household_code
trip_code_uc,,,,,
1065667000,2016-10-23,2,4239361,190.0,2054802
1065667001,2016-12-20,9850,0,NaN,8648515
1065667002,2016-11-14,6920,3727506,730.0,30230428
1065667003,2016-08-10,6920,3001662,605.0,2071112
1065667004,2016-04-25,5851,0,NaN,2076738


In [8]:
# Proportion of unnumerotated stores
print(f"Proportion of unnumerotated stores : {round(len(trips[trips.store_code_uc==0])/len(trips),4)*100}% (data droped, over {len(trips)} trips)")

Proportion of unnumerotated stores : 48.68% (data droped, over 10745635 trips)


In [9]:
explore = trips[trips.store_code_uc==0]

In [10]:
explore.groupby('retailer_code').count().describe()

,purchase_date,store_code_uc,store_zip3,household_code
count,814.000000,814.000000,814.0,814.000000
mean,6425.955774,6425.955774,0.0,6425.955774
std,22940.367216,22940.367216,0.0,22940.367216
min,1.000000,1.000000,0.0,1.000000
25%,32.000000,32.000000,0.0,32.000000
50%,322.500000,322.500000,0.0,322.500000
75%,2664.750000,2664.750000,0.0,2664.750000
max,246742.000000,246742.000000,0.0,246742.000000


In [ ]:
s=0
for retailer in trips[trips.store_code_uc!=0].retailer_code.unique():
    if retailer in explore.retailer_code.unique():
        s+=1
print(s)

## Getting the store state

In [11]:
zip_to_state = pd.read_table('zip_prefixes.txt', header=0, names=['zip3','state', 'distib_center', 'towns'])[['zip3', 'state']]

non_null = trips[trips.store_code_uc != 0]
non_null = non_null.merge(zip_to_state, left_on='store_zip3', right_on='zip3')[['purchase_date', 'retailer_code', 'store_code_uc', 'state', 'store_zip3', 'household_code']].rename(columns={'state': 'store_state'})
non_null

,purchase_date,retailer_code,store_code_uc,store_state,store_zip3,household_code
0,2016-10-23,2,4239361,PA,190.0,2054802
1,2016-03-20,24,254521,PA,190.0,30367513
2,2016-02-24,120,1021442,PA,190.0,30730891
3,2016-06-14,120,4301315,PA,190.0,30201210
4,2016-11-16,2,977112,PA,190.0,8646012
...,...,...,...,...,...,...
5514902,2016-12-24,45,1996462,VT,53.0,8286051
5514903,2016-01-09,4901,3520966,VT,53.0,8306752
5514904,2016-01-19,34,1997671,VT,53.0,8620167
5514905,2016-12-28,45,1996462,VT,53.0,8286051


## Getting the store county - mode

In [12]:
##
non_null_merged = non_null.merge(panelist, left_on='household_code', right_on='Household_Cd')[['retailer_code', 'store_code_uc', 'store_state', 'store_zip3', 'household_state', 'household_county_fips', 'household_county', 'household_zip3']]
non_null_merged

,retailer_code,store_code_uc,store_state,store_zip3,household_state,household_county_fips,household_county,household_zip3
0,2,4239361,PA,190.0,PA,42045,DELAWARE,190
1,6905,2890557,PA,190.0,PA,42045,DELAWARE,190
2,6905,2890557,PA,190.0,PA,42045,DELAWARE,190
3,2,4239361,PA,190.0,PA,42045,DELAWARE,190
4,6920,7620551,PA,190.0,PA,42045,DELAWARE,190
...,...,...,...,...,...,...,...,...
5514902,6920,7847066,WY,823.0,WY,56007,CARBON,823
5514903,272,2723868,WY,823.0,WY,56007,CARBON,823
5514904,6920,7847066,WY,823.0,WY,56007,CARBON,823
5514905,6920,7847066,WY,823.0,WY,56007,CARBON,823


In [13]:
# The store county is assumed to be the one where the maximum of households that visited this store comes form.

# Fisrt, we delete all the housolds that do not come from the store state.

before_deletion = len(non_null_merged)
non_null_state = non_null_merged[non_null_merged.household_state == non_null_merged.store_state] # TOUT RENOMMER
suppression_rate_state = 1-len(non_null_state)/before_deletion
print(f"Taux de suppression : {round(suppression_rate_state,4)*100}% (proportion de foyers qui ne vont pas dans un store de leur Etat)")

Taux de suppression : 2.75% (proportion de foyers qui ne vont pas dans un store de leur Etat)


In [14]:
# Then, we delete all the housolds that do not come from the store zip3 zone.

non_null_zip3 = non_null_state[non_null_state.household_zip3 == non_null_state.store_zip3]
suppression_rate_zip3 = 1-len(non_null_zip3)/len(non_null_state)
suppression_rate = 1 - (1 - suppression_rate_state) * (1 - suppression_rate_zip3)
print(f"Taux de suppression : {round(suppression_rate, 4)*100}% (proportion de foyers qui ne vont pas dans un store de leur zip3 zone)")

Taux de suppression : 16.97% (proportion de foyers qui ne vont pas dans un store de leur zip3 zone)


In [10]:
suppression_rate = 1 - (1 - suppression_rate_state) * (1 - suppression_rate_zip3)
print(f"Taux de suppression : {suppression_rate} (proportion de foyers qui ne vont pas dans un store de leur zip3 zone)")

Taux de suppression : 0.16967357745107936 (proportion de foyers qui ne vont pas dans un store de leur zip3 zone)


In [15]:
df = non_null_zip3

In [16]:
from random import choice

def my_mode(self):
    return choice(list(pd.Series.mode(self)))

In [17]:
# Then, we select the mode :
store_loc = df.groupby(['store_code_uc'])[['retailer_code', 'store_state', 'household_county_fips', 'household_county']].agg(my_mode).rename(columns={'household_county_fips': 'guessed_store_county_fips', 'household_county': 'guessed_store_county'})
store_loc.head()


,retailer_code,store_state,guessed_store_county_fips,guessed_store_county
store_code_uc,,,,
236,363,MN,27123,RAMSEY
540,4914,NC,37183,WAKE
557,3997,IL,17031,COOK
588,848,CA,6037,LOS ANGELES
920,290,FL,12099,PALM BEACH


In [14]:
def my_max(x):
    try:
        return max(x)
    except:
        return x
def my_min(x):
    try:
        return min(x)
    except:
        return x
def my_sum(x):
    try:
        return sum(x)
    except:
        return x
def my_len(x):
    try:
        return len(x)
    except:
        return 1


In [29]:
stat = df.groupby(['store_code_uc'])[['household_county_fips', 'household_county']].agg(pd.Series.value_counts)
stat['max_obs'] = stat.household_county.apply(my_max)
stat['nb_obs'] = stat.household_county.apply(my_sum)
stat['max_freq'] = stat.max_obs / stat.nb_obs
stat['distinct_counties'] = stat.household_county.apply(my_len)
stat['criteria'] = stat.max_obs >= 4 / 3 * (stat.nb_obs - (stat.distinct_counties + 2) - stat.nb_obs)

NameError: name 'my_max' is not defined

In [ ]:
stat.head()

In [16]:
stat['nb_min'] = stat.household_county.apply(my_min)
stat['criteria'] = ((stat.max_obs >= 3 * (stat.nb_obs - (stat.distinct_counties - 2)*stat.nb_min - stat.max_obs)) & (stat.distinct_counties!=1)) | ((stat.distinct_counties==1) & (stat.nb_obs>=4))


In [17]:
1 - (stat.criteria.sum()/len(stat))

0.16077797826360618

In [18]:
len(stat)-stat.criteria.sum()

7663

In [ ]:
import matplotlib.pyplot as plt

fig, axis = plt.subplots(4,2, figsize=(20,20))
fig.suptitle("Statistical analysis of the county choice", fontsize='xx-large')

axis[0][0].hist(stat.nb_obs, bins=30);
axis[0][0].set_title(f"Number of trips per store (after deletion of wrong states and zip3 zones)")
axis[0][0].set_xlabel("frequency")
axis[0][0].set_ylabel("nb of obsrveations (trips)")

axis[0][1].hist(stat.nb_obs[stat.nb_obs<30], bins=30);
axis[0][1].set_title(f"Number of trips per store (zoom)")
axis[0][1].set_xlabel("frequency")
axis[0][1].set_ylabel("nb of obserations (trips)")

axis[1][0].hist(stat.distinct_counties.to_list())
axis[1][0].set_title(f"Distinct counties observed for one store")
axis[1][0].set_xlabel("nb of counties")
axis[1][0].set_ylabel("nb of stores")

axis[1][1].hist(stat.max_freq.to_list(), align='left')
axis[1][1].set_title(f"Frequency of the mode county")
axis[1][1].set_xlabel("frequency")
axis[1][1].set_ylabel("nb of stores")

axis[2][0].scatter(stat.nb_obs[~stat.criteria], stat.max_freq[~stat.criteria], marker='.', color='red')
axis[2][0].scatter(stat.nb_obs[stat.criteria], stat.max_freq[stat.criteria], marker='.', color='green')
axis[2][0].set_title(f"Mode frequency vs nb of trips")
axis[2][0].set_xlabel("nb of observations (trips)")
axis[2][0].set_ylabel("mode frequency")

axis[2][1].scatter(stat.nb_obs[(~stat.criteria)&(stat.max_freq<0.8)], stat.max_freq[(~stat.criteria)&(stat.max_freq<0.8)], marker='.', color='red')
axis[2][1].scatter(stat.nb_obs[(stat.criteria)&(stat.max_freq<0.76)], stat.max_freq[(stat.criteria)&(stat.max_freq<0.76)], marker='.', color='green')
axis[2][1].set_title(f"Mode frequency vs nb of trips (zoom)")
axis[2][1].set_xlabel("nb of observations (trips)")
axis[2][1].set_ylabel("mode frequency")

axis[3][0].scatter(stat.nb_obs[(~stat.criteria)&(stat.nb_obs<50)], stat.max_freq[(~stat.criteria)&(stat.nb_obs<50)], marker='.', color='red')
axis[3][0].scatter(stat.nb_obs[(stat.criteria)&(stat.nb_obs<50)], stat.max_freq[(stat.criteria)&(stat.nb_obs<50)], marker='.', color='green')
axis[3][0].set_title(f"Mode frequency vs nb of trips (zoom)")
axis[3][0].set_xlabel("nb of observations (trips)")
axis[3][0].set_ylabel("mode frequency")

axis[3][1].scatter(stat.nb_obs[(stat.criteria)&(stat.max_freq>0.9)&(stat.nb_obs<100)], stat.max_freq[(stat.criteria)&(stat.max_freq>0.9)&(stat.nb_obs<100)], marker='.', color='green')
axis[3][1].scatter(stat.nb_obs[(~stat.criteria)&(stat.max_freq>0.9)&(stat.nb_obs<100)], stat.max_freq[(~stat.criteria)&(stat.max_freq>0.9)&(stat.nb_obs<100)], marker='.', color='red')
axis[3][1].set_title(f"Mode frequency vs nb of trips (zoom)")
axis[3][1].set_xlabel("nb of observations (trips)")
axis[3][1].set_ylabel("mode frequency")

plt.show()

In [ ]:
fig, axis = plt.subplots(7,1, figsize=(20,70))

for i in range(0,7):
    axis[i].scatter(stat.nb_obs[stat.distinct_counties==i+1][~stat.criteria], stat.max_freq[stat.distinct_counties==i+1][~stat.criteria], marker='.')


In [ ]:
plt.scatter(stat.nb_obs[stat.nb_obs<100], stat.max_freq[stat.nb_obs<100], marker='.');

In [20]:
trips.head()

,purchase_date,retailer_code,store_code_uc,store_zip3,household_code
trip_code_uc,,,,,
1065667000,2016-10-23,2,4239361,190.0,2054802
1065667001,2016-12-20,9850,0,NaN,8648515
1065667002,2016-11-14,6920,3727506,730.0,30230428
1065667003,2016-08-10,6920,3001662,605.0,2071112
1065667004,2016-04-25,5851,0,NaN,2076738


In [21]:
store_loc.head()

,retailer_code,store_state,guessed_store_county_fips,guessed_store_county
store_code_uc,,,,
236,363,MN,27123,RAMSEY
540,4914,NC,37183,WAKE
557,3997,IL,17031,COOK
588,848,CA,6037,LOS ANGELES
920,290,FL,12099,PALM BEACH


In [18]:
trips_loc = trips[trips.store_code_uc != 0].reset_index().merge(store_loc.reset_index(), on=['retailer_code', 'store_code_uc']).set_index('trip_code_uc')
trips_loc.head()

,purchase_date,retailer_code,store_code_uc,store_zip3,household_code,store_state,guessed_store_county_fips,guessed_store_county
trip_code_uc,,,,,,,,
1065667000,2016-10-23,2,4239361,190.0,2054802,PA,42045,DELAWARE
1065643502,2016-06-30,2,4239361,190.0,8431892,PA,42045,DELAWARE
1065702055,2016-03-08,2,4239361,190.0,30520933,PA,42045,DELAWARE
1065704006,2016-03-21,2,4239361,190.0,2054802,PA,42045,DELAWARE
1065723840,2016-01-17,2,4239361,190.0,31289665,PA,42045,DELAWARE


In [22]:
trips_loc = trips_loc.reset_index().merge(stat, on='store_code_uc')


NameError: name 'stat' is not defined

In [ ]:
trips_loc.head()

In [ ]:
trips_loc[trips_loc.criteria]

## UPC description

In [23]:
upc_descr = pd.read_table('../../Nielsen/nielsen_extracts/HMS/Master_Files/Latest/products.tsv', encoding = "ISO-8859-1")[['upc', 'upc_ver_uc', 'product_group_code', 'product_group_descr']]
upc_descr = upc_descr.merge(upc_descr.groupby('upc')['product_group_descr'].nunique(), on='upc').rename(columns={'product_group_descr_y': 'nb_of_groups_per_upc', 'product_group_descr_x': 'product_group_descr'})
upc_descr = upc_descr[upc_descr.nb_of_groups_per_upc == 1][['upc', 'upc_ver_uc', 'product_group_code', 'product_group_descr']]

In [25]:
upc_descr = upc_descr[upc_descr.product_group_code.apply(str)!='nan']
upc_descr = upc_descr.astype({'product_group_code': int})
print(f"Purchases - suppression rate of upc's with no group : {round(len(upc_descr) /len(purchases), 4)*100} %")

Purchases - suppression rate of upc's with no group : 549.99 %


In [26]:
purchases = purchases.merge(upc_descr, on=['upc', 'upc_ver_uc'])

In [ ]:
purchases.head()

In [28]:
trips_loc.head()

,purchase_date,retailer_code,store_code_uc,store_zip3,household_code,store_state,guessed_store_county_fips,guessed_store_county
trip_code_uc,,,,,,,,
1065667000,2016-10-23,2,4239361,190.0,2054802,PA,42045,DELAWARE
1065643502,2016-06-30,2,4239361,190.0,8431892,PA,42045,DELAWARE
1065702055,2016-03-08,2,4239361,190.0,30520933,PA,42045,DELAWARE
1065704006,2016-03-21,2,4239361,190.0,2054802,PA,42045,DELAWARE
1065723840,2016-01-17,2,4239361,190.0,31289665,PA,42045,DELAWARE


## Aggregating the data

In [27]:
## Merging :

prices = purchases.merge(trips_loc, on='trip_code_uc')[['trip_code_uc', 'purchase_date', 'retailer_code', 'store_code_uc', 'store_state', 'guessed_store_county', 'guessed_store_county_fips', 'upc', 'upc_ver_uc', 'product_group_code', 'product_group_descr', 'upc_price']]
prices['purchase_month'] = prices.purchase_date.dt.month

In [ ]:
prices.head()

In [28]:
avg = pd.DataFrame(prices.groupby(['retailer_code', 'store_state', 'guessed_store_county', 'guessed_store_county_fips', 'purchase_month', 'product_group_descr']).mean()[['upc_price']])

In [48]:
std = pd.DataFrame(prices.groupby(['retailer_code', 'store_state', 'guessed_store_county', 'guessed_store_county_fips', 'purchase_month', 'product_group_code', 'product_group_descr']).std()[['upc_price']]).rename(columns={'upc_price': 'upc_price_std'})

In [52]:
std.head()

upc_price_std
retailer_code store_state guessed_store_county guessed_store_county_fips purchase_month product_group_code product_group_descr                
2             DE          KENT                 10001                     1              1503               CARBONATED BEVERAGES       0.091924
                                                                                        4001               FRESH PRODUCE              0.288675
                                                                         2              1503               CARBONATED BEVERAGES       0.200475
                                                                                        1506               CRACKERS                        NaN
                                                                                        4001               FRESH PRODUCE              0.542245

In [56]:
avg_prices = avg.merge(std, left_index=True, right_index=True)

In [57]:
avg_prices.head(50)

upc_price  \
retailer_code store_state guessed_store_county guessed_store_county_fips purchase_month product_group_descr              product_group_code              
2             DE          KENT                 10001                     1              CARBONATED BEVERAGES             1503                 1.315000   
                                                                                        FRESH PRODUCE                    4001                 3.823333   
                                                                         2              CARBONATED BEVERAGES             1503                 1.559167   
                                                                                        CRACKERS                         1506                 3.000000   
                                                                                        FRESH PRODUCE                    4001                 3.396000   
                                                                         3              CARBONATED BEVERAGES             1503                 1.156250   
                                                                                        FRESH PRODUCE                    4001                 3.495000   
                                                                         4              CARBONATED BEVERAGES             1503                 1.591538   
                                                                                        FRESH PRODUCE                    4001                 3.247500   
                                                                                        MILK                             2506                 3.430000   
                                                                                        PERSONAL SOAP AND BATH ADDITIVES 4508                 7.990000   
                                                                         5              CARBONATED BEVERAGES             1503                 1.381818   
                                                                                        ICE CREAM, NOVELTIES             2005                 2.270000   
                                                                         6              CARBONATED BEVERAGES             1503                 1.391667   
                                                                                        CEREAL                           1005                 3.390000   
                                                                                        DESSERTS, GELATINS, SYRUP        1008                 2.000000   
                                                                                        JUICE, DRINKS - CANNED, BOTTLED  507                  2.290000   
                                                                                        PAPER PRODUCTS                   4507                 7.490000   
                                                                         7              PET FOOD                         508                  1.250000   
                                                                         8              BREAD AND BAKED GOODS            1501                 2.490000   
                                                                                        CARBONATED BEVERAGES             1503                 1.100000   
                                                                                        CEREAL                           1005                 2.990000   
                                                                                        CHEESE                           2502                 2.390000   
                                                                                        FRESH PRODUCE                    4001                 2.990000   
                                                                                        PERSONAL SOAP AND BATH ADDITIVES 4508            

In [30]:
ines = pd.read_table('../../Nielsen/nielsen_extracts/HMS/Master_Files/Latest/retailers.tsv')

In [32]:
ines[ines.channel_type=='Discount Store']

,retailer_code,channel_type
1151,6901,Discount Store
1152,6902,Discount Store
1153,6903,Discount Store
1154,6904,Discount Store
1155,6905,Discount Store
1156,6906,Discount Store
1157,6907,Discount Store
1158,6908,Discount Store
1159,6909,Discount Store
1160,6910,Discount Store


In [33]:
len(ines[ines.channel_type=='Discount Store'])

26